# No Globals Decorator

The purpose of this notebook is to show how to create a decorator to ensure a function does not erroneously read from global variables. This ensures that the function does not use variables from the global scope, which can avoid many common bugs.

Important notes:

* If done naively, the local scope will have no modules or functions. This is very frustrating, as you would need to import everything again

* We can explicitly keep the global scope's modules and functions. However, this implementation only stores the modules and functions at the time of the function definition, so be sure to have everything you need by there!

## Create `no_globals` decorator

In [1]:
import types
no_globals = lambda function: types.FunctionType(
    function.__code__,

    # Keep modules and functions in local scope
    {global_name: global_val
     for global_name, global_val in globals().items()
     if isinstance(global_val, types.ModuleType)
     or hasattr(global_val, '__call__')
    }
)

no_globals_bad = lambda function: types.FunctionType(
    function.__code__,
    
    # Completely empty local scope
    {}
)

## Compare `no_globals` to alternatives

In [2]:
# Be sure to include necessary imports before creating function!
import numpy as np

# BEST
@no_globals
def test_no_globals(x, repeat):
    return np.array([x]*repeat)

@no_globals
def test_no_globals_typo(x_typo, repeat_typo):
    return np.array([x]*repeat)

In [3]:
# BAD BASELINE 1: Use globals
def test_with_globals(x, repeat):
    return np.array([x]*repeat)

def test_with_globals_typo(x_typo, repeat_typo):
    return np.array([x]*repeat)

# BAD BASELINE 2: No globals, modules, or functions at all
@no_globals_bad
def test_no_globals_bad(x, repeat):
    return np.array([x]*repeat)

@no_globals_bad
def test_no_globals_bad_typo(x_typo, repeat_typo):
    return np.array([x]*repeat)

In [4]:
# Create global variables with the same variable name
x = 1
repeat = 2

## Show working `no_globals`

In [5]:
# WORKING
print("WORKING: CORRECT OUTPUT")
print(f"test_no_globals(x, repeat) = {test_no_globals(x, repeat)}")
print(f"test_no_globals(5, 10) = {test_no_globals(5, 10)}")

WORKING: CORRECT OUTPUT
test_no_globals(x, repeat) = [1 1]
test_no_globals(5, 10) = [5 5 5 5 5 5 5 5 5 5]


In [6]:
# WORKING
print("WORKING: ERROR FROM TYPO")
print(f"test_no_globals_typo(x, repeat) = {test_no_globals_typo(x, repeat)}")
print(f"test_no_globals_typo(5, 10) = {test_no_globals_typo(5, 10)}")

WORKING: ERROR FROM TYPO


NameError: name 'x' is not defined

## Show not working `with_globals`

In [7]:
# WORKING
print("WORKING: CORRECT OUTPUT")
print(f"test_with_globals(x, repeat) = {test_with_globals(x, repeat)}")
print(f"test_with_globals(5, 10) = {test_with_globals(5, 10)}")

WORKING: CORRECT OUTPUT
test_with_globals(x, repeat) = [1 1]
test_with_globals(5, 10) = [5 5 5 5 5 5 5 5 5 5]


In [8]:
# NOT WORKING
print("NOT WORKING: BUG FROM TYPO")
print(f"test_with_globals_typo(x, repeat) = {test_with_globals_typo(x, repeat)}")
print(f"test_with_globals_typo(5, 10) = {test_with_globals_typo(5, 10)}")

NOT WORKING: BUG FROM TYPO
test_with_globals_typo(x, repeat) = [1 1]
test_with_globals_typo(5, 10) = [1 1]


## Show not working `no_globals_bad`

In [9]:
# NOT WORKING
print("NOT WORKING: MISSING NP IMPORT")
print(f"test_no_globals_bad(x, repeat) = {test_no_globals_bad(x, repeat)}")
print(f"test_no_globals_bad(5, 10) = {test_no_globals_bad(5, 10)}")

NOT WORKING: MISSING NP IMPORT


NameError: name 'np' is not defined

In [10]:
# NOT WORKING
print("NOT WORKING: MISSING NP IMPORT")
print(f"test_no_globals_bad_typo(x, repeat) = {test_no_globals_bad_typo(x, repeat)}")
print(f"test_no_globals_bad_typo(5, 10) = {test_no_globals_bad_typo(5, 10)}")

NOT WORKING: MISSING NP IMPORT


NameError: name 'np' is not defined